# The 64-dim *Ring* example in the [GBS paper](http://proceedings.mlr.press/v118/jia20a.html)

*last tested with bayesfast commit 8d6efa1*

**(Erratum)** There's a typo in eq 7 of the GBS paper, noted by Eric Chuu in October 2020. The likelihood function should be:

$\ln\mathcal{L}=-\frac{(x_n^2+x_1^2-a)^2}{b}
-\sum_{i=1}^{n-1}\frac{(x_i^2+x_{i+1}^2-a)^2}{b},
\quad a=2,
\quad b=1,
\quad n=64$

In [1]:
import bayesfast as bf
import numpy as np
from threadpoolctl import threadpool_limits
threadpool_limits(1) # TODO: implement a bayesfast global thread controller

In [2]:
D = 64 # number of dims
a = 2.
b = 1.
lower = np.full(D, -5.) # lower bound of the prior
upper = np.full(D, 5.) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff)) # normalization of the flat prior

def logp(x):
    x2 = x * x
    x2s = np.concatenate((x2[..., -1:], x2, x2[..., :1]), axis=-1)
    return -np.sum((x2s[..., :-2] + x2s[..., 1:-1] - a)**2 / b, axis=-1) - const

def grad(x):
    x2 = x * x
    x2s = np.concatenate((x2[..., -1:], x2, x2[..., :1]), axis=-1)
    return -(2 * (x2s[..., :-2] + x2s[..., 1:-1] - a) / b + 
             2 * (x2s[..., 1:-1] + x2s[..., 2:] - a) / b) * 2 * x

In [3]:
bf.utils.random.set_generator(64) # set up the global random number generator
bf.utils.parallel.set_backend(8) # set up the global parallel backend
den = bf.DensityLite(logp=logp, grad=grad, input_size=D, input_scales=bound,
                     hard_bounds=True)
sample_trace = {'n_chain': 8, 'n_iter': 2500, 'n_warmup': 1000}
rec = bf.Recipe(density=den, sample={'sample_trace': sample_trace},
                post={'evidence_method': 'GBS'})

In [4]:
rec.run()


 *** StaticSample: returning the #0 SampleStep. *** 



/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #4 : divide by zero encountered in log
  return np.sum(np.log(np.abs(self.to_original_grad(x_trans))),
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #5 : divide by zero encountered in log
  return np.sum(np.log(np.abs(self.to_original_grad(x_trans))),
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:992: RuntimeWarning:  CHAIN #4 : divide by zero encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog
/global/u2/h/hejia/bayesfast/bayesfast/samplers/nuts.py:120: RuntimeWarning:  CHAIN #4 : overflow encountered in exp
  p_accept = min(1, np.exp(-energy_change))
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:992: RuntimeWarning:  CHAIN #5 : divide by zero encountered in true_divide
  _grad += self.to_original_grad2(x) / _tog
/global/u2/h/hejia/bayesfast/bayesfast/core/density.py:183: RuntimeWarning:  CHAIN #6 : divide by zero encountered in l

 CHAIN #3 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.17 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.48 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.51 seconds. (warmup)
 CHAIN #2 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.82 seconds. (warmup)
 CHAIN #4 : sampling proceeding [ 500 / 2500 ], last 500 samples used 8.98 seconds. (warmup)
 CHAIN #7 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.14 seconds. (warmup)
 CHAIN #5 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.21 seconds. (warmup)
 CHAIN #6 : sampling proceeding [ 500 / 2500 ], last 500 samples used 9.73 seconds. (warmup)
 CHAIN #3 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 7.98 seconds. (warmup)
 CHAIN #1 : sampling proceeding [ 1000 / 2500 ], last 500 samples used 7.81 seconds. (warmup)
 CHAIN #0 : sampling proceeding [ 1000 / 2500 ], last 500 samples us

/global/homes/h/hejia/.conda/envs/hejia@cori-3/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/global/homes/h/hejia/.conda/envs/hejia@cori-3/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/global/homes/h/hejia/.conda/envs/hejia@cori-3/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/global/homes/h/hejia/.conda/envs/hejia@cori-3/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: Convergenc


 ***** PostStep finished. ***** 



In [5]:
rec.get()._fields

('samples',
 'weights',
 'weights_trunc',
 'logp',
 'logq',
 'logz',
 'logz_err',
 'x_p',
 'x_q',
 'logp_p',
 'logq_q',
 'trace_p',
 'trace_q',
 'n_call',
 'x_max',
 'f_max')

In [6]:
rec.get().logz, rec.get().logz_err # fiducial value: logz = -114.492

(-114.47258569257664, 0.06493140280021414)